In [ ]:
!pip install image-classifiers==1.0.0b1
!pip install keras_applications --no-deps
!pip install tensorflow==2.10

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
print(tf.__version__)

2.10.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Load Adience dataset

In [ ]:
fold0 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold0')
fold1 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold1')
fold2 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold2')
fold3 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold3')
fold4 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold4')
train = fold0.concatenate(fold1)
train = train.concatenate(fold2)
val = fold3
test = fold4

### Load base model pretrained on CELEBA

In [ ]:
base_model = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')

- Last 40 layers are output layers, we want to freeze all layers except the last 10 layers (not including 40 output layers). <br> Hence, we make all layers except the last 50 layers non-trainable

In [ ]:
for layer in base_model.layers[:-50]:
  layer.trainable = False

### Create model for Adience based on pretrained model

In [ ]:
def create_model(base_model):
  ''' 
  Model pretrained on CELEBA has 40 outputs, each corrsponds to presence of a facial feature. 
  We continue from its output, reshape it to a vector of (40, 1) representing features extracted from the image.
  This 'feature vector' will be used to train our age_gender model.
  After a Dropout to prevent overfitting, we create two outputs, one each for age and gender

  Args:
        base_model: ResNet34 model pretrained on CELEBA
        
  Returns:
        new model used to train age_gender
  '''

  img = tf.keras.Input(shape=(224,224,3))

  # add data augmentation
  img = tf.keras.layers.RandomFlip(mode='horizontal_and_vertical')(img)
  img = tf.keras.layers.RandomRotation(0.1)(img)
  img = tf.keras.layers.RandomContrast(0.1)(img)

  # pass the image through base model trained on CELEBA
  base_outputs = base_model(img)

  '''
  base_outputs = [[[0.3], [0.2], ..., [last_in_batch]], # first element is outputs of first feature, in shape of (batch_size, 1) 
                  [[0.5], [0.8], ..., [last_in_batch]], # second element is outputs of second feature, in shape of (batch_size, 1) 
                  ...]
  we reshape each element in base_outputs to have shape (batch_size,)
  base_outputs become: [[0.3, 0.2, ..., last_in_batch], 
                        [0.5, 0.8, ..., last_in_batch], 
                        ...]
  now each column is a feature vector of length 40. Hence, we transpose it so that each feature vector takes a row
  '''
  base_outputs = tf.convert_to_tensor([tf.reshape(i, [-1]) for i in base_outputs])
  base_outputs = tf.transpose(base_outputs)

  # add Dropout to prevent overfitting
  base_outputs = tf.keras.layers.Dropout(0.6)(base_outputs)
  
  # two outputs, each with regularization to prevent overfitting
  gender_branch = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='gender_output')(base_outputs)
  age_branch = tf.keras.layers.Dense(8, activation='softmax', kernel_regularizer=l2(0.1), name='age_output')(base_outputs)
  
  model = tf.keras.Model(inputs = img,
                         outputs = [gender_branch, age_branch])
  
  return model

In [ ]:
# create model and compile it
model = create_model(base_model)
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0003),
    loss={'gender_output': 'binary_crossentropy', 'age_output': 'categorical_crossentropy'},
    metrics={'gender_output': 'accuracy', 'age_output': 'accuracy'},
)

# model summary
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_1 (Functional)           [(None, 1),          21322993    ['input_10[0][0]']               
                                 (None, 1),                                                       
                                 (None, 1),                                                       
                                 (None, 1),                                                       
                                 (None, 1),                                                 

In [ ]:
# create new metrics: mean of age + gen accuracy for train and val
class MeanAccCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    logs['mean_age_gen_acc'] = (logs.get('gender_output_accuracy') + logs.get('age_output_accuracy'))/2
    logs['mean_val_age_gen_acc'] = (logs.get('val_gender_output_accuracy') + logs.get('val_age_output_accuracy'))/2

In [ ]:
# save the model with best mean val_age_gen_acc
file_path = "/content/drive/MyDrive/data/saved_model/celeba_age_gen_base.h5"

checkpoint = ModelCheckpoint(file_path, monitor='mean_val_age_gen_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
hist = model.fit(train.shuffle(1024).batch(64),
                 epochs=20, 
                 validation_data = val.shuffle(1024).batch(64),
                 callbacks = [
                     EarlyStopping(monitor = 'val_loss', patience=5),
                     MeanAccCallback(),
                     checkpoint,])

Epoch 1/20
118/118 [==============================] - ETA: 0s - loss: 4.0346 - gender_output_loss: 0.7002 - age_output_loss: 2.0732 - gender_output_accuracy: 0.5456 - age_output_accuracy: 0.1995
Epoch 1: mean_val_age_gen_acc improved from -inf to 0.41254, saving model to /content/drive/MyDrive/data/saved_model/celeba_age_gen_base.h5
118/118 [==============================] - 30s 201ms/step - loss: 4.0346 - gender_output_loss: 0.7002 - age_output_loss: 2.0732 - gender_output_accuracy: 0.5456 - age_output_accuracy: 0.1995 - val_loss: 3.7305 - val_gender_output_loss: 0.6718 - val_age_output_loss: 1.9632 - val_gender_output_accuracy: 0.5811 - val_age_output_accuracy: 0.2440 - mean_age_gen_acc: 0.3725 - mean_val_age_gen_acc: 0.4125
Epoch 2/20
118/118 [==============================] - ETA: 0s - loss: 3.5779 - gender_output_loss: 0.6541 - age_output_loss: 1.9615 - gender_output_accuracy: 0.6130 - age_output_accuracy: 0.2487
Epoch 2: mean_val_age_gen_acc improved from 0.41254 to 0.46590, savi

### Graphs for loss and accuracy against epochs

### Save and load trained model

In [ ]:
model = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/celeba_age_gen_base.h5')

In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_1 (Functional)           [(None, 1),          21322993    ['input_10[0][0]']               
                                 (None, 1),                                                       
                                 (None, 1),                                                       
                                 (None, 1),                                                       
                                 (None, 1),                                                 

### Evaluate trained model on test data

In [ ]:
metrics = model.evaluate(test.batch(64), return_dict = True)

36/36 [==============================] - 25s 409ms/step - loss: 2.5050 - gender_output_loss: 0.5462 - age_output_loss: 1.7446 - gender_output_accuracy: 0.7384 - age_output_accuracy: 0.3543


In [ ]:
metrics

{'loss': 2.5049638748168945,
 'gender_output_loss': 0.5461651086807251,
 'age_output_loss': 1.744632601737976,
 'gender_output_accuracy': 0.7384135723114014,
 'age_output_accuracy': 0.3542780876159668}

In [ ]:
test_ds = test.batch(1)
preds = model.predict(test_ds)

2244/2244 [==============================] - 25s 11ms/step


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def gender_metrics(preds, ds):
  gender_true = []
  for i in ds.as_numpy_iterator():
    # take label dict
    label_dict = i[1] 
    # append all values with key 'gender_output' to a list
    gender = label_dict['gender_output'][0] 
    # append all gender to a list
    gender_true.append(gender) 
  gender_preds = np.where(preds[0].ravel()>0.5, 1, 0)
  return {'accuracy': accuracy_score(gender_true, gender_preds), 
          'f1': f1_score(gender_true, gender_preds), 
          'precision': precision_score(gender_true, gender_preds), 
          'recall': recall_score(gender_true, gender_preds)}

In [ ]:
gender_metrics(preds, test_ds)

{'accuracy': 0.7384135472370766,
 'f1': 0.7664146438519698,
 'precision': 0.7317629179331308,
 'recall': 0.8045112781954887}